In [1]:
%load_ext autoreload
%autoreload 2

In [17]:
import coinbase

In [18]:
api.client.aclose()

<coroutine object AsyncClient.aclose at 0x000001ADF105A040>

In [19]:
api = coinbase.Market()

In [20]:
r = await api.coins()
r

<Response [200 OK]>